# Pediatric Model Without Transfer

Utilizes empty model

Import all needed in this notebook

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoImageProcessor, ResNetForImageClassification
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from torch.utils.data import DataLoader, TensorDataset, Subset, random_split
from torch.optim.lr_scheduler import CosineAnnealingLR
import numpy as np
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from tqdm import tqdm
import zipfile
import torch.nn.functional as F
from tabulate import tabulate

## Model Creation

Same code for the model

In [2]:
class CustomCNNModel(nn.Module):
    def __init__(self):
        super(CustomCNNModel, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(128 * 28 * 28, 512) 
        self.fc2 = nn.Linear(512, 256)  
        self.fc3 = nn.Linear(256, 2)
        
        self.dropout = nn.Dropout(p=0.5) 

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) 
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        
        x = x.view(-1, 128 * 28 * 28)
        
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        
        return x

All layers will remain trainable since we start from fresh.

## Dataset Retrieval

Retrieve dataset and compute X (data images in tensor format) and y (binary encoded label)

In [3]:
# with zipfile.ZipFile("original_data.zip", 'r') as zip_ref:
#     zip_ref.extractall("")

In [4]:
transform = transforms.Compose([transforms.ToTensor()])

dataset = datasets.ImageFolder(root="../pediatric_dataset/preprocessed_medium", transform=transform)

test_size = 0.75
random_seed = 42

labels = [dataset[i][1] for i in range(len(dataset))]

print(f"Complete Dataset: {len(dataset)} images")

Complete Dataset: 400 images


## Training Methods

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
def train_model(model, train_loader, test_loader, criterion, optimizer, scheduler, num_epochs=10):
    model.train()
    
    train_losses, train_accuracies = [], []
    test_losses, test_accuracies = [], []
    test_precisions, test_recalls, test_f1s = [], [], []

    print(f"Device using for training: {device}")
    model = model.to(device)

    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        with tqdm(train_loader, unit="batch", desc=f"Epoch [{epoch+1}/{num_epochs}]") as tbar:
            for inputs, labels in tbar:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = 100 * correct / total
        train_losses.append(epoch_loss)
        train_accuracies.append(epoch_accuracy)

        test_loss, test_accuracy, precision, recall, f1 = evaluate_model(model, test_loader, criterion)
        test_losses.append(test_loss)
        test_accuracies.append(test_accuracy)
        test_precisions.append(precision)
        test_recalls.append(recall)
        test_f1s.append(f1)
        
        scheduler.step()

        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_accuracy:.2f}%, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")

    return train_losses, train_accuracies, test_losses, test_accuracies, test_precisions, test_recalls, test_f1s


def k_fold_cross_validation_with_split(model_class, dataset, criterion, optimizer_class, scheduler_class, k=5, num_epochs=10, batch_size=32):
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

    fold_results = []
    
    labels = [dataset[i][1] for i in range(len(dataset))]

    print(f"Running {k}-Fold Cross-Validation...")

    for fold, (rest_folds_indices, one_fold_indices) in enumerate(skf.split(dataset, labels)):
        print(f"Fold {fold + 1}/{k}: Training on {len(one_fold_indices)} samples, Testing on {len(rest_folds_indices)} samples")

        train_subset = Subset(dataset, one_fold_indices)
        test_subset = Subset(dataset, rest_folds_indices)
        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_subset, batch_size=batch_size, shuffle=False)

        model = model_class()
        optimizer = optimizer_class(model.parameters())
        scheduler = scheduler_class(optimizer)

        train_losses, train_accuracies, test_losses, test_accuracies, test_precisions, test_recalls, test_f1s = train_model(
            model, train_loader, test_loader, criterion, optimizer, scheduler, num_epochs
        )

        fold_results.append({
            'fold': fold + 1,
            'train_losses': train_losses,
            'train_accuracies': train_accuracies,
            'test_losses': test_losses,
            'test_accuracies': test_accuracies,
            'test_precisions': test_precisions,
            'test_recalls': test_recalls,
            'test_f1s': test_f1s
        })
        
    return fold_results

def evaluate_model(model, test_loader, criterion):
    model.eval()
    test_losses, correct, total = [], 0, 0
    all_labels, all_preds = [], []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_losses.append(loss.item())

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

    test_loss = sum(test_losses) / len(test_loader)
    test_accuracy = 100 * correct / total

    precision = precision_score(all_labels, all_preds, average="weighted", zero_division=0)
    recall = recall_score(all_labels, all_preds, average="weighted", zero_division=0)
    f1 = f1_score(all_labels, all_preds, average="weighted", zero_division=0)

    return test_loss, test_accuracy, precision, recall, f1

## Transfer Training

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer_class = lambda params: optim.Adam(params, lr=1e-3, weight_decay=1e-2)
scheduler_class = lambda optimizer: CosineAnnealingLR(optimizer, T_max=15)
model_class = CustomCNNModel
num_epochs = 15

results = k_fold_cross_validation_with_split(model_class, dataset, criterion, optimizer_class, scheduler_class, k=4, num_epochs=num_epochs, batch_size=32)

Running 4-Fold Cross-Validation...
Fold 1/4: Training on 100 samples, Testing on 300 samples
Device using for training: cuda


Epoch [1/15]: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.43batch/s]


Epoch [1/15], Train Loss: 0.7174, Train Accuracy: 51.00%, Test Loss: 0.7030, Test Accuracy: 50.00%


Epoch [2/15]: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.98batch/s]


Epoch [2/15], Train Loss: 0.7004, Train Accuracy: 51.00%, Test Loss: 0.6901, Test Accuracy: 50.00%


...

Epoch [13/15], Train Loss: 0.3677, Train Accuracy: 84.00%, Test Loss: 0.4379, Test Accuracy: 78.00%


Epoch [14/15]: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.94batch/s]


Epoch [14/15], Train Loss: 0.5089, Train Accuracy: 85.00%, Test Loss: 0.4263, Test Accuracy: 80.67%


Epoch [15/15]: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.92batch/s]


Epoch [15/15], Train Loss: 0.3407, Train Accuracy: 84.00%, Test Loss: 0.4245, Test Accuracy: 80.33%


## Evaluation

In [8]:
def compute_confidence_interval(data):
    mean_val = np.mean(data)
    std_val = np.std(data)
    n = len(data)
    margin_of_error = 1.96 * (std_val / np.sqrt(n))
    
    lower_bound = mean_val - margin_of_error
    upper_bound = mean_val + margin_of_error
    
    return mean_val, lower_bound, upper_bound

In [10]:
# for result in results:
#     print(f"Fold {result['fold']}:")
#     print(f"  Train Accuracy: {result['train_accuracies'][-1]:.2f}%")
#     print(f"  Test Accuracy: {result['test_accuracies'][-1]:.2f}%")
#     print(f"  Test Precision: {result['test_precisions'][-1] * 100:.2f}%")
#     print(f"  Test Recall: {result['test_recalls'][-1] * 100:.2f}%")
#     print(f"  Test F1-score: {result['test_f1s'][-1] * 100:.2f}%")

train_losses = [fold['train_losses'][-1] for fold in results]
train_accuracies = [fold['train_accuracies'][-1] for fold in results]
test_losses = [fold['test_losses'][-1] for fold in results]
test_accuracies = [fold['test_accuracies'][-1] for fold in results]
test_precisions = [fold['test_precisions'][-1] for fold in results]
test_recalls = [fold['test_recalls'][-1] for fold in results]
test_f1s = [fold['test_f1s'][-1] for fold in results]

print(f"Test Accuracies: {test_accuracies}")
print(f"Test Precisions: {test_precisions}")
print(f"Test Rccuracies: {test_recalls}")
print(f"Test F1s: {test_f1s}")

avg_train_loss, ci_train_loss_low, ci_train_loss_high = compute_confidence_interval(train_losses)
avg_train_accuracy, ci_train_acc_low, ci_train_acc_high = compute_confidence_interval(train_accuracies)
avg_test_loss, ci_test_loss_low, ci_test_loss_high = compute_confidence_interval(test_losses)
avg_test_accuracy, ci_test_acc_low, ci_test_acc_high = compute_confidence_interval(test_accuracies)
avg_test_precision, ci_test_prec_low, ci_test_prec_high = compute_confidence_interval(test_precisions)
avg_test_recall, ci_test_rec_low, ci_test_rec_high = compute_confidence_interval(test_recalls)
avg_test_f1, ci_test_f1_low, ci_test_f1_high = compute_confidence_interval(test_f1s)

headers = ["Metric", "Mean", "95% CI Lower", "95% CI Upper"]
data = [
    ["Training Loss", f"{avg_train_loss:.4f}", f"{ci_train_loss_low:.4f}", f"{ci_train_loss_high:.4f}"],
    ["Test Loss", f"{avg_test_loss:.4f}", f"{ci_test_loss_low:.4f}", f"{ci_test_loss_high:.4f}"],
    ["Training Accuracy (%)", f"{avg_train_accuracy:.2f}", f"{ci_train_acc_low:.2f}", f"{ci_train_acc_high:.2f}"],
    ["Test Accuracy (%)", f"{avg_test_accuracy:.2f}", f"{ci_test_acc_low:.2f}", f"{ci_test_acc_high:.2f}"],
    ["Test Precision (%)", f"{avg_test_precision * 100:.2f}", f"{ci_test_prec_low * 100:.2f}", f"{ci_test_prec_high * 100:.2f}"],
    ["Test Recall (%)", f"{avg_test_recall * 100:.2f}", f"{ci_test_rec_low * 100:.2f}", f"{ci_test_rec_high * 100:.2f}"],
    ["Test F1-score (%)", f"{avg_test_f1 * 100:.2f}", f"{ci_test_f1_low * 100:.2f}", f"{ci_test_f1_high * 100:.2f}"]
]

print("\nOverall Performance with 95% Confidence Intervals:")
print(tabulate(data, headers=headers, tablefmt="grid"))

Test Accuracies: [83.0, 82.66666666666667, 77.0, 80.33333333333333]
Test Precisions: [0.8490011750881316, 0.8295373027259685, 0.7804838635209381, 0.8063973063973063]
Test Rccuracies: [0.83, 0.8266666666666667, 0.77, 0.8033333333333333]
Test F1s: [0.8276541819205857, 0.8262883613202084, 0.7678305050527708, 0.8028404344193818]

Overall Performance with 95% Confidence Intervals:
+-----------------------+---------+----------------+----------------+
| Metric                |    Mean |   95% CI Lower |   95% CI Upper |
+=======================+=========+================+================+
| Training Loss         |  0.3354 |         0.2114 |         0.4595 |
+-----------------------+---------+----------------+----------------+
| Test Loss             |  0.4198 |         0.352  |         0.4875 |
+-----------------------+---------+----------------+----------------+
| Training Accuracy (%) | 84.75   |        80.51   |        88.99   |
+-----------------------+---------+----------------+---------